In [1]:
# Import required libraries for this section
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import cv2

import torch
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler


from torch import nn,optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

70/30 for train/test

Random crops of size 227×227 were generated from inside the 256×256 images to feed the first layer of AlexNet. Note that the paper mentions the network inputs to be 224×224, but that is a mistake and the numbers make sense with 227×227 instead.

In [2]:
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of data set to use as test
test_size = 0.3

transform = transforms.Compose([ transforms.CenterCrop(1000), transforms.Resize((299,299)),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

data_set = dset.ImageFolder(root="data",transform=transform)
dataloader = torch.utils.data.DataLoader(data_set, batch_size=4,shuffle=True,num_workers=2)

# obtain training indices that will be used for test
num_data = len(data_set)
indices = list(range(num_data))
np.random.shuffle(indices)
split = int(np.floor(test_size * num_data))
train_idx, test_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
test_sampler  = SubsetRandomSampler(test_idx)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(data_set, batch_size=batch_size,
                                           sampler = train_sampler, num_workers=num_workers)
test_loader  = torch.utils.data.DataLoader(data_set, batch_size=batch_size, 
                                           sampler = test_sampler, num_workers=num_workers)

classes = ('blurry','clear')

In [3]:
model = models.inception_v3(pretrained=True, aux_logits=True)
criterion = nn.CrossEntropyLoss()

x = torch.randn(2, 3, 299, 299)
target = torch.empty(2, dtype=torch.long).random_(1000)

output = model(x)
loss = criterion(output[0], target)
loss.backward()

print(model.fc.weight.grad)

tensor([[4.9358e-05, 3.0247e-04, 3.9718e-05,  ..., 4.4720e-04, 1.6878e-04,
         3.3124e-04],
        [9.9329e-05, 8.4195e-04, 7.9929e-05,  ..., 1.2066e-03, 4.6981e-04,
         8.5308e-04],
        [4.3932e-04, 4.7727e-04, 3.5352e-04,  ..., 1.0682e-03, 2.6632e-04,
         1.1772e-03],
        ...,
        [4.3312e-05, 3.3904e-04, 3.4853e-05,  ..., 4.8921e-04, 1.8919e-04,
         3.4953e-04],
        [2.6504e-04, 3.0737e-04, 2.1328e-04,  ..., 6.7001e-04, 1.7151e-04,
         7.2572e-04],
        [2.4870e-04, 2.8715e-04, 2.0012e-04,  ..., 6.2702e-04, 1.6023e-04,
         6.7996e-04]])


In [4]:
# Use GPU if it's available
from collections import OrderedDict
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.inception_v3(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
fc = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.fc = fc

criterion = nn.CrossEntropyLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

device = "cpu"
model.to(device);


In [5]:
epochs = 5
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in train_loader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps[0], labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        

    test_loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
         for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
                    
            test_loss += batch_loss.item()
                    
                    # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
    print(f"Epoch {epoch+1}/{epochs}.. "
           f"Train loss: {running_loss:.3f}.. "
           f"Test loss: {test_loss/len(test_loader):.3f}.. "
           f"Test accuracy: {accuracy/len(test_loader):.3f}")
    
    with open('inception.csv','a') as f:
        f.write(f"Epoch {epoch+1}/{epochs}.. "
           f"Train loss: {running_loss:.3f}.. "
           f"Test loss: {test_loss/len(test_loader):.3f}.. "
           f"Test accuracy: {accuracy/len(test_loader):.3f}")
        f.write("\n")
        
    running_loss = 0
    model.train()

Epoch 1/5.. Train loss: 19.036.. Test loss: 0.523.. Test accuracy: 0.758
Epoch 2/5.. Train loss: 15.510.. Test loss: 0.493.. Test accuracy: 0.815
Epoch 3/5.. Train loss: 13.721.. Test loss: 0.497.. Test accuracy: 0.735
Epoch 4/5.. Train loss: 12.431.. Test loss: 0.425.. Test accuracy: 0.854
Epoch 5/5.. Train loss: 13.182.. Test loss: 0.463.. Test accuracy: 0.819
